In [2]:
import pandas as pd

In [9]:
github_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv'
url = f'{github_url}?raw=1'
df = pd.read_csv(url)

df = df.iloc[:300]
print(df)

                                            answer_llm  \
0    You can sign up for the course by visiting the...   
1    You can sign up using the link provided in the...   
2    Yes, there is an FAQ for the Machine Learning ...   
3    The context does not provide any specific info...   
4    To structure your questions and answers for th...   
..                                                 ...   
295  An alternative way to load the data using the ...   
296  You can directly download the dataset from Git...   
297  You can fetch data for homework using the `req...   
298  If the status code is 200 when downloading dat...   
299  If the file download fails when using the requ...   

                                           answer_orig  document  \
0    Machine Learning Zoomcamp FAQ\nThe purpose of ...  0227b872   
1    Machine Learning Zoomcamp FAQ\nThe purpose of ...  0227b872   
2    Machine Learning Zoomcamp FAQ\nThe purpose of ...  0227b872   
3    Machine Learning Zoomcamp 

In [10]:
from sentence_transformers import SentenceTransformer
model_name = 'multi-qa-mpnet-base-dot-v1'
embedding_model = SentenceTransformer(model_name)
answer_llm = df.iloc[0].answer_llm

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

You try to use a model that was created with version 3.0.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:
embedding = embedding_model.encode(answer_llm)
print(embedding[0])

-0.42244658


In [15]:
import numpy as np

In [16]:
evaluations = []

for _, row in df.iterrows():
    embedding_origin = embedding_model.encode(row.answer_orig)
    embedding_llm = embedding_model.encode(row.answer_llm)
    evaluations.append(np.dot(embedding_origin, embedding_llm))

In [18]:
np.percentile(evaluations, 75)

31.674307823181152

In [21]:
evaluations_2 = []

for _, row in df.iterrows():
    embedding_origin = embedding_model.encode(row.answer_orig)
    embedding_llm = embedding_model.encode(row.answer_llm)
    
    norm_origin = np.sqrt((embedding_origin * embedding_origin).sum())
    norm_llm = np.sqrt((embedding_llm * embedding_llm).sum())
    evaluations_2.append(np.dot(embedding_origin / norm_origin, embedding_llm / norm_llm))


In [22]:
np.percentile(evaluations_2, 75)

0.836234837770462

In [23]:
!pip install rouge

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Obtaining dependency information for rouge from https://files.pythonhosted.org/packages/32/7c/650ae86f92460e9e8ef969cc5008b24798dcf56a9a8947d04c78f550b3f5/rouge-1.0.1-py3-none-any.whl.metadata

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [24]:
from rouge import Rouge
rouge_scorer = Rouge()

r = df.iloc[10]
scores = rouge_scorer.get_scores(r['answer_orig'], r['answer_llm'])[0]

f_score = scores['rouge-1']['f']

In [25]:
f_score

0.45454544954545456

In [28]:
f_rouge_1 = scores['rouge-1']['f']
f_rouge_2 = scores['rouge-2']['f']
f_rouge_l = scores['rouge-l']['f']
average_f = (f_rouge_1 + f_rouge_2 + f_rouge_l) / 3
print(average_f)

0.36500136000136507


In [31]:
from rouge import Rouge
rouge_scorer = Rouge()
rouge_scores = []

for _, row in df.iterrows():
    scores = rouge_scorer.get_scores(row['answer_orig'], row['answer_llm'])[0]
    f_rouge_2 = scores['rouge-2']['f']
    rouge_scores.append(f_rouge_2)

rouge = pd.Series(rouge_scores)
print(rouge.mean())

0.20696501983423318
